# Correlation analysis between the Bitcoin currency and Twitter

This project consists of a correlation analysis between the Bitcoin currency and tweets. In order to define the positiveness of a tweet (if the course of the bitcoin will go up or down), we realise a sentiment analysis of each tweet using the VADER algorithm. Finally we try to find a correlation between the two and we will make some machine learning to make predictions.

This notebook was written using Python 3.6.

# Sentiment analysis

We will first read all the tweets that we retrieved with the API in the TwitterExtraction notebook.

In [1]:
import json
import pandas as pd
import io

d = pd.read_csv('tweets_raw.csv')
d.head(5)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,Text,UserName,UserFollowerCount,RetweetCount,CreatedAt
0,995987428166000642,RT @CryptoAmb: Cryptocurrency is not far from ...,Mark Brown,402,1,Mon May 14 11:21:22 +0000 2018
1,995987423271235585,"RT @icopool: 10,000 XRP Giveaway is now on! 10...",Billy Blocks,333,25,Mon May 14 11:21:20 +0000 2018
2,995987420058275841,Want to switch your mining between different c...,MaxiMine,404,0,Mon May 14 11:21:20 +0000 2018
3,995987412550606849,Day Trading: 2 Manuscripts: Absolute Beginners...,Blockchain,17876,0,Mon May 14 11:21:18 +0000 2018
4,995987411472670723,Arduino: The Comprehensive Beginner's Guide To...,Blockchain,17876,0,Mon May 14 11:21:18 +0000 2018


### VADER
Here we do the sentiment analysis to calculate the sentiment score for each tweet with the VADER algorithm. Then we add the number of retweets of each tweet as a weight to the score.

Don't forget to install VaderSentiment `pip install VaderSentiment`!

In [3]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
#neg = []
#neu = []
#pos = []
compound = []
for i,s in enumerate(d['Text']):
    vs = analyzer.polarity_scores(s)
    #print("{:-<65} {}".format(s, str(vs)))
    #neg.append(vs["neg"])
    #neu.append(vs["neu"])
    #pos.append(vs["pos"])
    compound.append(vs["compound"])
#d["neg"] = pd.Series(neg)
#d["neu"] = pd.Series(neu)
#d["pos"] = pd.Series(pos)
d["compound"] = pd.Series(compound)

d.head(5)

,ID,Text,UserName,UserFollowerCount,RetweetCount,CreatedAt,compound
0,995987428166000642,RT @CryptoAmb: Cryptocurrency is not far from ...,Mark Brown,402,1,Mon May 14 11:21:22 +0000 2018,0.0000
1,995987423271235585,"RT @icopool: 10,000 XRP Giveaway is now on! 10...",Billy Blocks,333,25,Mon May 14 11:21:20 +0000 2018,0.8381
2,995987420058275841,Want to switch your mining between different c...,MaxiMine,404,0,Mon May 14 11:21:20 +0000 2018,0.0772
3,995987412550606849,Day Trading: 2 Manuscripts: Absolute Beginners...,Blockchain,17876,0,Mon May 14 11:21:18 +0000 2018,0.0000
4,995987411472670723,Arduino: The Comprehensive Beginner's Guide To...,Blockchain,17876,0,Mon May 14 11:21:18 +0000 2018,0.2500


In [18]:
scores = []
for i,s in d.iterrows():
    scores.append(s["compound"] * ((s["RetweetCount"]+1)/1000) * ((s["UserFollowerCount"]+1)/1000))
    #print(s["compound"] * ((s["RetweetCount"]+1)/1000) * ((s["UserFollowerCount"]+1)/1000))
d["Scores"] = pd.Series(scores)

TypeError: must be str, not int

In [ ]:
import matplotlib.pyplot as plt
#d['CreatedAt'] = pd.to_datetime(d['CreatedAt'])
#d.index = d['CreatedAt']
#d2 = d.groupby(d.index.hour).mean()
#print(d2)
plt.plot_date(d["CreatedAt"], d["Scores"], ls='-', marker='o')
plt.show()

## Retrieve the Bitcoin currency
Here we use the Kraken API.

TODO Do we retrieve data from multiple API (maybe they don't have the same prices).

TODO Maybe we should make a daily prevision

In [7]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime

In [10]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [11]:
def get_quandl_data(quandl_id):
    '''Download and cache Quandl dataseries'''
    cache_path = '{}.pkl'.format(quandl_id).replace('/','-')
    try:
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df

In [12]:
btc_usd_price_kraken = get_quandl_data('BCHARTS/KRAKENUSD')


Cached BCHARTS/KRAKENUSD at BCHARTS-KRAKENUSD.pkl


In [13]:
btc_usd_price_kraken.head()


,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2014-01-07,874.67040,892.06753,810.00000,810.00000,15.622378,13151.472844,841.835522
2014-01-08,810.00000,899.84281,788.00000,824.98287,19.182756,16097.329584,839.156269
2014-01-09,825.56345,870.00000,807.42084,841.86934,8.158335,6784.249982,831.572913
2014-01-10,839.99000,857.34056,817.00000,857.33056,8.024510,6780.220188,844.938794
2014-01-11,858.20000,918.05471,857.16554,899.84105,18.748285,16698.566929,890.671709


In [19]:
btc_trace = go.Scatter(x=btc_usd_price_kraken.index, y=btc_usd_price_kraken['Weighted Price'])
py.iplot([btc_trace])

## Correlation analysis
Here we make the correlation analysis between Bitcoin currency and tweets.

We can test our correlation hypothesis using the Pandas corr() method, which computes a Pearson correlation coefficient for each column in the dataframe against each other column.

In [15]:
d.corr(btc_usd_price_kraken.reindex(d.index, method='pad'))

TypeError: Cannot compare type 'Timestamp' with type 'int'

In [17]:
d.head(5)

,ID,Text,UserName,UserFollowerCount,RetweetCount,CreatedAt,compound
0,995987428166000642,RT @CryptoAmb: Cryptocurrency is not far from ...,Mark Brown,402,1,Mon May 14 11:21:22 +0000 2018,0.0000
1,995987423271235585,"RT @icopool: 10,000 XRP Giveaway is now on! 10...",Billy Blocks,333,25,Mon May 14 11:21:20 +0000 2018,0.8381
2,995987420058275841,Want to switch your mining between different c...,MaxiMine,404,0,Mon May 14 11:21:20 +0000 2018,0.0772
3,995987412550606849,Day Trading: 2 Manuscripts: Absolute Beginners...,Blockchain,17876,0,Mon May 14 11:21:18 +0000 2018,0.0000
4,995987411472670723,Arduino: The Comprehensive Beginner's Guide To...,Blockchain,17876,0,Mon May 14 11:21:18 +0000 2018,0.2500


# TODO Check twythonstreamer?

Stream the new tweets without limit?? It would be nice for realtime predictions

In [ ]:
from twython import TwythonStreamer

class MyStreamer(TwythonStreamer):
    def on_success(self, data):
        if 'text' in data:
            print(data['text'])

    def on_error(self, status_code, data):
        print(status_code)

        # Want to stop trying to get data because of the error?
        # Uncomment the next line!
        # self.disconnect()
OAUTH_TOKEN = "3459248236-0XPtHldG3ou6BfpTwaKWnOL2ywFk2niQekLwE7K"
OAUTH_TOKEN_SECRET = "08Vy2wuOkp7AmuC3rbjCHFJ94MLG2sWqdvGQtoiXmkVKr"

stream = MyStreamer(APP_KEY, APP_SECRET,
                    OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
stream.statuses.filter(track='bitcoin')